Responsabilidades:
- Simular la respuesta al escalón de Gcl(s) con los valores hallados.
- Implementar el PID en el script de TempLABUdeA.
- Realizar la prueba práctica, recolectar los datos de temperatura y control.
- Comparar los datos experimentales con la simulación (gráficas, %OS, Ts).

Entregables:
- Código de simulación y control.
- Archivos CSV de los datos experimentales.
- Gráficas comparativas: simulación vs experimento.